En este archivo, se explica e implementa el proceso de calcular las distancias a los vehículos que se encuentran en el mismo carril que nuestro coche.

In [ ]:
def process_video(video_path, output_path, model, device, resize_factor=0.5, score_threshold=0.5):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return False
    else:
        # Create Writer with same properties that the original video
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)
        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
        out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        print(f"Processing video... Resolution: {width}x{height}, FPS: {fps}")
        
        frame_count = 0
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break # Video ended
            
            # Convert the image to Tensor format
            frame_tensor = F.to_tensor(frame).to(device)

            # Process the frame and write it in the ouput route
            pred_masks, pred_boxes, pred_labels = get_predictions(frame_tensor, model, score_threshold)
            frame_processed = draw_predictions2(frame_tensor, pred_masks, pred_boxes, pred_labels)
            out.write(frame_processed)
            
            # Show the frame processing in real time
            #cv2.imshow("Processed video", frame_processed)
            
            frame_count += 1
            if frame_count % 50 == 0:
                print(f"{frame_count} frames processed...")

            # Exit if press 'q'
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # Liberar recursos
        cap.release()
        out.release()
        cv2.destroyAllWindows()
        return True